In [3]:
import sqlite3 
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
# from lib import MongoHandler, WeatherGetter
conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

In [4]:
cur.execute("""SELECT DISTINCT Date, HomeTeam, FTHG, AwayTeam, FTAG, FTR
                FROM teams_in_matches
                JOIN unique_teams
                USING(unique_team_id)
                JOIN matches
                USING (match_id)
                WHERE season IN (SELECT season
                                    FROM matches 
                                    WHERE season == '2011')
            """)

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df = df.sort_values(by=['FTR'])
df.head()

,Date,HomeTeam,FTHG,AwayTeam,FTAG,FTR
0,2012-03-31,Nurnberg,0,Bayern Munich,1,A
673,2011-10-01,Everton,0,Liverpool,2,A
676,2011-10-01,Wolves,1,Newcastle,2,A
677,2011-10-02,Bolton,1,Chelsea,5,A
687,2011-10-15,Wigan,1,Bolton,3,A


In [5]:
df['Date'] = pd.to_datetime(df['Date'])

In [6]:
from datetime import date
from time import mktime

Timestamps = []

for x in df['Date']:
    Timestamps.append(mktime(x.timetuple()))

In [7]:
df['Timestamps'] = Timestamps
df['Timestamps'] = df['Timestamps'].astype(int)
df.head()

,Date,HomeTeam,FTHG,AwayTeam,FTAG,FTR,Timestamps
0,2012-03-31,Nurnberg,0,Bayern Munich,1,A,1333148400
673,2011-10-01,Everton,0,Liverpool,2,A,1317423600
676,2011-10-01,Wolves,1,Newcastle,2,A,1317423600
677,2011-10-02,Bolton,1,Chelsea,5,A,1317510000
687,2011-10-15,Wigan,1,Bolton,3,A,1318633200


In [8]:
import requests
import json

r = requests.get('https://api.darksky.net/forecast/958870d7f05d6ce07081ed0698de9eca/13.404954,52.520008,1572548834')
r.json()

{'latitude': 13.404954,
 'longitude': 52.520008,
 'timezone': 'Asia/Aden',
 'currently': {'time': 1572548834,
  'summary': 'Humid',
  'icon': 'clear-night',
  'precipIntensity': 0.0002,
  'precipProbability': 0.01,
  'precipType': 'rain',
  'temperature': 82.27,
  'apparentTemperature': 88.62,
  'dewPoint': 74.26,
  'humidity': 0.77,
  'pressure': 1009.2,
  'windSpeed': 10.72,
  'windGust': 12.14,
  'windBearing': 238,
  'cloudCover': 0.12,
  'uvIndex': 0,
  'visibility': 10,
  'ozone': 265.1},
 'hourly': {'summary': 'Humid throughout the day.',
  'icon': 'clear-day',
  'data': [{'time': 1572469200,
    'summary': 'Humid',
    'icon': 'clear-night',
    'precipIntensity': 0,
    'precipProbability': 0,
    'temperature': 81.37,
    'apparentTemperature': 86.85,
    'dewPoint': 73.84,
    'humidity': 0.78,
    'pressure': 1009.7,
    'windSpeed': 11.32,
    'windGust': 12.08,
    'windBearing': 217,
    'cloudCover': 0.16,
    'uvIndex': 0,
    'visibility': 10,
    'ozone': 268.8},
   

In [9]:
#AWAY WINS

cur.execute("""SELECT DISTINCT match_id, AwayTeam 
                FROM teams_in_matches
                JOIN unique_teams
                USING(unique_team_id)
                JOIN matches
                USING (match_id)
                WHERE season IN (SELECT season
                                    FROM matches 
                                    WHERE season == '2011')
               AND
               ftr = 'A'   
            """)

away_df = pd.DataFrame(cur.fetchall())
away_df.columns = [i[0] for i in cur.description]
away_df = away_df.groupby(away_df['AwayTeam']).count()

In [10]:
#HOME WINS
cur.execute("""SELECT DISTINCT match_id, HomeTeam 
                FROM teams_in_matches
                JOIN unique_teams
                USING(unique_team_id)
                JOIN matches
                USING (match_id)
                WHERE season IN (SELECT season
                                    FROM matches 
                                    WHERE season == '2011')
               AND
               ftr = 'H'   
            """)

home_df = pd.DataFrame(cur.fetchall())
home_df.columns = [i[0] for i in cur.description]
home_df = home_df.groupby(home_df['HomeTeam']).count()